In [90]:
!pip install async-generator==1.10
!pip install attrs==22.2.0
!pip install beautifulsoup4==4.11.2
!pip install bs4==0.0.1
!pip install certifi==2022.12.7
!pip install cffi==1.15.1
!pip install exceptiongroup==1.1.0
!pip install h11==0.14.0
!pip install idna==3.4
!pip install install==1.3.5
!pip install numpy==1.24.2
!pip install outcome==1.2.0
!pip install pandas==1.5.3
!pip install pycparser==2.21
!pip install PySocks==1.7.1
!pip install python-dateutil==2.8.2
!pip install pip install pytz==2022.7.1
!pip install PyYAML==6.0
!pip install selenium==4.8.0
!pip install six==1.16.0
!pip install sniffio==1.3.0
!pip install sortedcontainers==2.4.0
!pip install soupsieve==2.3.2.post1
!pip install trio==0.22.0
!pip install trio-websocket==0.9.2
!pip install tqdm
!pip install urllib3==1.26.14
!pip install wsproto==1.2.0

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is 

In [92]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
import yaml
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import pandas as pd
import traceback

In [102]:
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(options=options)

In [103]:
with open('config.yaml') as f:
    var = yaml.safe_load(f)

In [104]:
driver.get("https://linkedin.com/uas/login")

In [105]:
driver.find_element("xpath", '//*[@id="username"]').send_keys(var['username'])
driver.find_element("xpath", '//*[@id="password"]').send_keys(var['password'])
time.sleep(1)

In [106]:
driver.find_element(
    "xpath", '//*[@id="organic-div"]/form/div[3]/button').click()

In [111]:
# go to job page, the url here is data scientist job posting. Replace with the job you want to scrape.
driver.get("https://www.linkedin.com/jobs/search/?currentJobId=3569702242&f_TPR=r86400&geoId=103644278&keywords=software%20engineer&location=United%20States&refresh=true")

In [112]:
links = []
print('Links collecting now.')
try:
    for page in tqdm(range(2, 10)): 

        jobs_block = driver.find_element(By.CLASS_NAME, 'jobs-search-results-list')
        jobs_list = jobs_block.find_elements(By.CLASS_NAME, 'jobs-search-results__list-item')
        for job in jobs_list:
            all_links = job.find_elements(By.TAG_NAME, 'a')
            for a in all_links:
                if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute('href') not in links:
                    links.append(a.get_attribute('href'))
                else:
                    pass
            driver.execute_script("arguments[0].scrollIntoView();", job)
        driver.find_element(
            "xpath", f"//button[@aria-label='Page {page}']").click()
except Exception:
    traceback.print_exc()
print('Found ' + str(len(links)) + ' links for job offers')

Links collecting now.


100%|██████████| 8/8 [01:00<00:00,  7.55s/it]

Found 200 links for job offers


In [113]:
# Create empty lists to store information
job_titles = []
company_names = []
company_locations = []
work_methods = []
post_dates = []
work_times = []
job_desc = []

i = 0
j = 1

for url in tqdm(links):
    time.sleep(4)
    try:
        driver.get(url)
        driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[2]/footer').click()
        time.sleep(2)
    except Exception:
        pass
    try:
        driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[4]/footer').click()
    except:
        pass

    try:
        job_titles.append(driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[1]/div/div/div[1]/h1').text)
    except Exception:
        job_titles.append('unknown')
    try:
        company_names.append(driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[1]/div/div/div[1]/div[1]/span[1]/span[1]').text)
    except Exception:
        company_names.append('unknown')
    try:
        company_locations.append(driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[1]/div/div/div[1]/div[1]/span[1]/span[2]').text)
    except Exception:
        company_locations.append('unknown')
    try:
        work_methods.append(driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[1]/div/div/div[1]/div[1]/span[1]/span[3]').text)
    except Exception:
        work_methods.append('unknown')
    try:
        post_dates.append(driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[1]/div/div/div[1]/div[1]/span[2]/span[1]').text)
    except Exception:
        post_dates.append('unknown')
    try:
        work_times.append(driver.find_element(
            "xpath", '/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[1]/div/div/div[1]/div[2]/ul/li[1]').text)
    except Exception:
        work_times.append('unknown')

    j += 1

100%|██████████| 200/200 [35:34<00:00, 10.67s/it]


In [115]:
df = pd.DataFrame(list(zip(job_titles, company_names,
                           company_locations, work_methods,
                           post_dates, work_times, links)),
                  columns=['job_title', 'company_name',
                           'company_location', 'work_method',
                           'post_date', 'work_time', 'job_links'])


In [119]:
df.to_excel('job_offers.xlsx', index=False)
df.head()

,job_title,company_name,company_location,work_method,post_date,work_time,job_links
0,Senior Associate Application Engineer (Full St...,Discover Financial Services,"Chicago, IL",On-site,1 hour ago,Full-time · Mid-Senior level,https://www.linkedin.com/jobs/view/3569701412/...
1,Software Engineer,Stealth Startup,San Francisco Bay Area,On-site,18 minutes ago,Full-time,https://www.linkedin.com/jobs/view/3569281540/...
2,Principal Software Full Stack Engineer,Workday,"Boulder, CO",Hybrid,21 hours ago,Full-time,https://www.linkedin.com/jobs/view/3545473585/...
3,Infrastructure Engineer,The Boston Dynamics AI Institute,"Cambridge, MA",On-site,15 hours ago,Full-time · Mid-Senior level,https://www.linkedin.com/jobs/view/3561171353/...
4,Software Engineer - (New Graduate Program),Thales,"Irvine, CA",Hybrid,15 hours ago,Full-time,https://www.linkedin.com/jobs/view/3568517854/...
